### We define a table schema which inherits most of the functionality from the database.

- auto generation of tables
- easy change the database

- since SQLite is so forgiving, pandas will check for correct dtypes and fail

### Known issues
- some prices display wrong in SQLite viewer, after querying they are fine
- the datatypes change, i guess that's how SQLite works

### Some special things about SQLite
All string datatypes in SQLite are converted to a TEXT datatype.  
If you try to specify a size for a string datatype, SQLite will ignore it.  
So long we have no performance loss we define them for convenience.

In [1]:
from mlrepricer import schemas, helper
from mlrepricer.database import SQLite
import pandas as pd

In [2]:
tableobject = schemas.pricemonitor(SQLite)()

we have useful properties like  
we will use some of those below

In [3]:
print(dir(tableobject))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_asin', '_booly', '_conn_data', '_datetimey', '_datey', '_floaty', '_inty', '_numericy', '_sellerid', '_textlong', '_textmiddle', '_textshort', '_timey', 'conn', 'createtable', 'dtypes', 'eventdate', 'if_exists', 'mapping', 'nullable', 'table']


In [4]:
tableobject.createtable?

Type:        property
String form: <property object at 0x7ff82e368d68>
Docstring:   Create  empty table if needed, with all columns, autoid, notnull.


In [5]:
tableobject.conn?

Type:        property
String form: <property object at 0x7ff82eb07c78>
Docstring:   Return a connection string you use like pandas.read_sql_table.


### Example how you can push data from a msg pack file into the database

In [7]:
df_from_msgpack = helper.load_dataframe('alldata')

In [8]:
# If you are not sure the database exists. You can always run this
tableobject.createtable

In [9]:
df_from_msgpack.to_sql(tableobject.table, tableobject.conn,
                  dtype=tableobject.dtypes,
                  if_exists='append', index=False)

In [10]:
# Now let's query that database
df = pd.read_sql_query("SELECT * FROM price_monitor", tableobject.conn, parse_dates=['time_changed'], index_col='ID')

In [11]:
df.dtypes

asin                          object
feedback                       int64
feedbackpercent                int64
instock                        int64
isbuyboxwinner                 int64
isfeaturedmerchant             int64
isprime                        int64
price                        float64
sellerid                      object
shipping_maxhours              int64
shipping_minhours              int64
time_changed          datetime64[ns]
dtype: object

### How the schema looks after tableobject.create
CREATE TABLE price_monitor (  
	"ID" INTEGER NOT NULL,  
	asin NVARCHAR(40) NOT NULL,  
	feedback INTEGER NOT NULL,  
	feedbackpercent INTEGER NOT NULL,  
	instock BOOLEAN NOT NULL,  
	isbuyboxwinner BOOLEAN NOT NULL,  
	isfeaturedmerchant BOOLEAN NOT NULL,  
	isprime BOOLEAN NOT NULL,  
	price DECIMAL(10, 2) NOT NULL,  
	sellerid NVARCHAR(40) NOT NULL,  
	shipping_maxhours INTEGER NOT NULL,  
	shipping_minhours INTEGER NOT NULL,  
	time_changed DATETIME NOT NULL,  
	PRIMARY KEY ("ID"),  
	CHECK (instock IN (0, 1)),  
	CHECK (isbuyboxwinner IN (0, 1)),  
	CHECK (isfeaturedmerchant IN (0, 1)),  
	CHECK (isprime IN (0, 1))  
);